In [2]:
%load_ext autoreload
%autoreload 2

import sys
import random
import goose
import numpy as np
import operator
import os

import textscrape as tscrape
import blogclassify

from collections import Counter
from pymongo import MongoClient

from nltk.tokenize import sent_tokenize
import nltk

import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import cross_val_score
from sklearn.externals import joblib


# Connect to MongoDB
client = MongoClient()
db = client.dsbc              
print(db.collection_names())         
poliblogs = client.dsbc.poliblogs

# Get GoogleAPI Key
keypath = os.environ["HOME"] + '/metis/secure/google_api_key.password'
google_api_key = tscrape.get_api_key(keypath)

def insert_posts_in_mongo(postdictlist, leaning, mongo_collection):
    blog_post_count = 0
    for post in postdictlist:
        post['blog_leaning'] = leaning
        mongo_collection.insert(post)
        blog_post_count += 1
    print("Number of blog posts inserted into Mongo: %i" % (blog_post_count))
    
def cross_val_scoring(pipeline, X, y):
    accuracy_scores = cross_val_score(pipeline, X, y, cv=5)
    print("Accuracy:\t %.3f" % np.mean(accuracy_scores))
    precision_scores = cross_val_score(pipeline, X, y, cv=5, scoring='precision')
    print("Precision:\t %.3f" % np.mean(precision_scores))
    recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
    print("Recall: \t %.3f" % np.mean(recall_scores))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[u'poliblogs', u'system.indexes']


In [5]:
d = []
if d: 
    print('Fail')

### Model Training and Testing

In [5]:
# Test and Train sets based on individual posts

min_num_sentences = 4

# Select "_id" values of blogs for training and testing samples
liberal_nums = list(poliblogs.find({ '$and':[{'blog_leaning': 'liberal'}, {'num_sentences': { '$gt': min_num_sentences }}]}, {'_id':1}))
conservative_nums = list(poliblogs.find({ '$and':[{'blog_leaning': 'conservative'}, {'num_sentences': { '$gt': min_num_sentences }}]}, {'_id':1}))
neutral_nums = list(poliblogs.find({ '$and':[{'$or':[{'blog_leaning': 'moderate'}, {'blog_leaning': 'review'}]}, {'num_sentences': { '$gt': min_num_sentences }}]}, {'_id':1})) 
min_num = min(len(liberal_nums), len(conservative_nums), len(neutral_nums))

liberal_nums_select = random.sample(liberal_nums, min_num)
conservative_nums_select = random.sample(conservative_nums, min_num)
neutral_nums_select = random.sample(neutral_nums, min_num)

s = liberal_nums_select + conservative_nums_select + neutral_nums_select
s = [p['_id'] for p in s]
train_nums, test_nums = train_test_split(s, train_size=0.9)
print(("Total Number of Posts found given constraints:", len(liberal_nums) + len(conservative_nums) + len(neutral_nums)))
print(("Total Number of Posts after taking even distribution of options:", len(s)))
print(("Number of training posts:", len(train_nums)))
print(("Number of testing posts: ", len(test_nums)))

# Actually build sets
train_posts = [p['post_content'] for thisid in train_nums for p in list(poliblogs.find({'_id': thisid}))]
train_answers = [p['blog_leaning'] for thisid in train_nums for p in list(poliblogs.find({'_id': thisid}))]
test_posts = [p['post_content'] for thisid in test_nums for p in list(poliblogs.find({'_id': thisid}))]
test_answers = [p['blog_leaning'] for thisid in test_nums for p in list(poliblogs.find({'_id': thisid}))]

train_answers = [t if t in ['conservative', 'liberal'] else 'neutral' for t in train_answers]
test_answers = [t if t in ['conservative', 'liberal'] else 'neutral' for t in test_answers]

Total Number of Posts found given constraints: 60429
Total Number of Posts after taking even distribution of options: 15420
Number of training posts: 13878
Number of testing posts:  1542


In [6]:
# Vectorize and build and test model
train_document_list = train_posts
train_answer_list = train_answers
test_document_list = test_posts
test_answer_list = test_answers

combo_document_list = train_document_list + test_document_list
combo_answer_list = train_answer_list + test_answer_list

kmax = 'all'

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,4), min_df=0.001, max_df=0.6)
vectorizer_bin = TfidfVectorizer(stop_words="english", ngram_range=(1,4), min_df=0.001, max_df=0.6, binary=True)
kselect = SelectKBest(k=kmax)
multimodel = MultinomialNB()
bernmodel = BernoulliNB()
gaussmodel = GaussianNB()
#############################################################################################
# multi_pipeline = Pipeline([('vectorizer', vectorizer), ('model', multimodel)])
# bern_pipeline = Pipeline([('vectorizer_bin', vectorizer_bin), ('model', bernmodel)])
# gauss_pipeline = Pipeline([('vectorizer', vectorizer), ('model', gaussmodel)])


# print "Testing Multinomial Model...."
# cross_val_scoring(multi_pipeline, combo_document_list, combo_answer_list)
# print

# print "Testing Bernoulli Model...."
# cross_val_scoring(bern_pipeline, combo_document_list, combo_answer_list)
# print

# print "Testing Gaussian Model...."
# cross_val_scoring(gauss_pipeline, combo_document_list, combo_answer_list)
# print

In [10]:
# One off model

doc_vectors_train = vectorizer.fit_transform(train_document_list)
doc_vectors_train = doc_vectors_train.toarray()
# doc_vectors_train = kselect.fit_transform(doc_vectors_train, train_answer_list)
multimodel.fit(doc_vectors_train, train_answer_list)


doc_vectors_test = vectorizer.transform(test_document_list)
doc_vectors_test = doc_vectors_test.toarray()
# doc_vectors_test = kselect.transform(doc_vectors_test)
predictions = multimodel.predict(doc_vectors_test)
# predictions = model.predict(doc_vectors_test.toarray())

accuracy_score =  sklearn.metrics.accuracy_score(test_answer_list, predictions)
print(("Accuracy Score: %.3f" % accuracy_score))
print((sklearn.metrics.classification_report(test_answer_list, predictions)))

classify = blogclassify.BlogClassify(vectorizer, multimodel, num_sentences=min_num_sentences)

Accuracy Score: 0.800
             precision    recall  f1-score   support

conservative       0.87      0.74      0.80       503
    liberal       0.78      0.80      0.79       506
    neutral       0.77      0.85      0.81       533

avg / total       0.80      0.80      0.80      1542



In [36]:
# Export Vectorizer and Model
# joblib.dump(vectorizer, 'vectorizer.pkl', compress=9)
joblib.dump(multimodel, 'model.pkl', compress=9)

['model.pkl']

In [44]:
print((doc_vectors_train.shape))

(9020, 35948)


### Now Apply Model to Other Blogs (Testing Continued)

In [26]:
# extremeliberal_posts (liberal)
# wrtdirection_posts (conservative)
# krugman_posts (liberal)
# friedman_posts (liberal)
# sergesblog_posts (conservative)
# flapolitics_posts (neutral)

post_list_names = [('extremeliberal_posts', 'liberal'), ('wrtdirection_posts', 'conservative'),
                   ('krugman_posts', 'liberal'), ('friedman_posts', 'liberal'), 
                   ('sergesblog_posts', 'conservative'), ('flapolitics_posts', 'neutral')
                    ]

for bn in post_list_names:
    print(bn)
    model_string = 'classification = classify.political_blog_model(' + bn[0] + ')'
    exec(model_string)
    print(classification)
    print()

('extremeliberal_posts', 'liberal')
Number of posts meeting criteria: 7
Counter({u'liberal': 5, u'neutral': 2})

('wrtdirection_posts', 'conservative')
Number of posts meeting criteria: 14
Counter({u'liberal': 5, u'conservative': 5, u'neutral': 4})

('krugman_posts', 'liberal')
Number of posts meeting criteria: 9
Counter({u'neutral': 8, u'liberal': 1})

('friedman_posts', 'liberal')
Number of posts meeting criteria: 11
Counter({u'conservative': 7, u'liberal': 3, u'neutral': 1})

('sergesblog_posts', 'conservative')
Number of posts meeting criteria: 6439
Counter({u'conservative': 3348, u'liberal': 2250, u'neutral': 841})

('flapolitics_posts', 'neutral')
Number of posts meeting criteria: 3670
Counter({u'neutral': 3254, u'liberal': 334, u'conservative': 82})



In [28]:
classification = classify.political_blog_model(dailykos_posts)
print(classification)

NameError: name 'dailykos_posts' is not defined

In [381]:
# political_blog_model(extremeliberal_posts)

In [33]:
feministing_posts = tscrape.get_articles_rss("http://feministing.com/feed")
cl = classify.political_blog_model(feministing_posts)
print(cl)

Number of posts recovered: 10
Number of posts meeting criteria: 1
Counter({u'liberal': 1})


### Create Lists of Blog Posts from Specific Blogs (do before testing using these above)

In [15]:
# sergesblog.blogspot.com (Conservative)
sergesblog_posts = tscrape.get_articles_blogger('http://sergesblog.blogspot.com', 
                                                    google_api_key, onlyposts=True)

In [16]:
# http://flapolitics.blogspot.com/ (Neutral)
flapolitics_posts = tscrape.get_articles_blogger('http://flapolitics.blogspot.com/', 
                                                    google_api_key, onlyposts=True)

In [17]:
# thewrtdirection.com
url_list = ['http://thewrtdirection.com/welcome-politically-correct-america/' 
                        'http://thewrtdirection.com/liberals-refuse-to-lead/', 
                        'http://thewrtdirection.com/should-states-decide-on-social-issues/', 
                        'http://thewrtdirection.com/truth-about-abortion/', 
                        'http://thewrtdirection.com/obama-is-dividing-america',
                        'http://thewrtdirection.com/obama-approval-rating-is-in-his-wife-still-likes-him/',
                        'http://thewrtdirection.com/i-think-the-founding-fathers-would-have-had-a-pistol-permit/',
                        'http://thewrtdirection.com/how-dare-you-have-a-problem-with-illegal-immigration/',
                        'http://thewrtdirection.com/stop-blaming-obama-for-his-lack-of-leadership-you-voted-for-him/',
                        'http://thewrtdirection.com/if-you-dont-support-entitlements-you-dont-care-about-people/', 
                        'http://thewrtdirection.com/if-our-country-is-liberal-why-dont-i-know-any-of-them/', 
                        'http://thewrtdirection.com/blog-tour-my-writing-process/', 
                        'http://thewrtdirection.com/how-a-book-can-purify-your-mind-and-your-drinking-water/', 
                        'http://thewrtdirection.com/how-to-overcome-writers-block-and-boost-productivity/', 
                        'http://thewrtdirection.com/7-writing-tools-that-will-help-you-write-like-a-pro/', 
                        'http://thewrtdirection.com/book-review-of-the-best-seller-vampires-in-the-lemon-grove/']
wrtdirection_posts = tscrape.get_articles_urls(url_list)

Number of urls to attempt: 15
Number of posts successfully scraped: 14


In [18]:
# extremeliberal.wordpress.com
extremeliberal_posts = ["""Once again, Jon Stewart and his staff reveal the craziness that is Fox News. If only other news outlets pushed back on their fellow “journalists”, but they are too busy worrying about President Obama taking a few days off from the hardest fucking job in the world, which even a lot of Republicans don’t care about. (this clip seems to be sped up a bit, sorry)""",
"""I have this fantasy that there are real journalists left in the world of cable news. But just like my other fantasies, they never seem to come true. (Insert Rimshot here)
Glenn Greenwald’s latest piece of “advocacy” journalism deals with events that started in 2002 and ended in 2008. It involves the NSA under President Bush spying on 5 prominent Americans who are Muslim. For the record, at the time the Cheney/Bush administration was selling their lies to the American people, I was marching against their march to war.
What was Glenn Greenwald thinking in 2002, when this spying began. From the preface to one of Glenn’s books, his own words…
I believed that Islamic extremism posed a serious threat to the country, and I wanted an aggressive response from our government. I was ready to stand behind President Bush and I wanted him to exact vengeance on the perpetrators and find ways to decrease the likelihood of future attacks. (emphasis mine)
Think about that for a minute. Greenwald was 36 years old at the time, according to my calculations. Not some young naive kid. Whenever he has tried to refute my pointing that out, he usually says something like, “everyone was doing it.” As my mother would occasionally say, if everyone jumped off a cliff, does that mean you should too?
More from Glenn Greenwald’s own keyboard…
During the following two weeks, my confidence in the Bush administration grew as the president gave a series of serious, substantive, coherent, and eloquent speeches that struck the right balance between aggression and restraint. And I was fully supportive of both the president’s ultimatum to the Taliban and the subsequent invasion of Afghanistan when our demands were not met. Well into 2002, the president’s approval ratings remained in the high 60 percent range, or even above 70 percent, and I was among those who strongly approved of his performance. [...]
Glenn “strongly approved of his (Bush’s) performance.” As I was screaming at my television as the propaganda poured out of the television, with conjecture, flag waving, minimization of dissenting opinions…”if you’re not with us, you’re against us.” Well, Glenn Greenwald was WITH President George W. Bush as he started his “aggressive response” and “exact(ed) vengeance on the perpetrators.”
I’m sure Glenn regrets writing that preface, just a little more of his love for President Bush.
Despite these doubts, concerns, and grounds for ambivalence, I had not abandoned my trust in the Bush administration. Between the president’s performance in the wake of the 9/11 attacks, the swift removal of the Taliban in Afghanistan, and the fact that I wanted the president to succeed, because my loyalty is to my country and he was the leader of my country, I still gave the administration the benefit of the doubt. I believed then that the president was entitled to have his national security judgment deferred to, and to the extent that I was able to develop a definitive view, I accepted his judgment that American security really would be enhanced by the invasion of this sovereign country.
I guess Glenn Greenwald made the cable TV rounds the other day after publishing his latest piece. Back to my fantasy. I was amusing myself by thinking of questions Chris L. Hayes might ask him on his show like…
1. In 2002, when this surveillance began, you were supporting President Bush and even put in the preface to your book that you were “ready to stand behind President Bush and I wanted him to exact vengeance on the perpetrators.” Do you feel any remorse for enabling the exact thing that you now decry?
2. Do you still believe that “American security really would be enhanced by the invasion of this sovereign country”, as you put in your book? What exactly were you thinking in 2002 when you wrote that?
3. Why is it that you “trusted”, “deferred to”, had “confidence in”, “strongly approved”, was “fully supportive” of Bush, gave Bush the “benefit of the doubt” and believed that President Bush “was entitled to have his national security judgement deferred to” BUT when a Democratic president comes into office, one who ended the Iraq War, ended the torture of innocent people, ended DADT, is ending the war in Afghanistan and hasn’t started any wars…why has that president had nothing but negative blog posts written about him. Glenn?
Those are just a few of the questions I fantasize about journalists asking of Glenn Greenwald.
Feel free to make your own conclusions about why Greenwald was so supportive of the administration that brought on most of the NSA abuses but has nothing but contempt for President Obama, one of the very few voices in the lead up to the Iraq war who was brave enough to stand up as a State Senator and say…
What I am opposed to is a dumb war. What I am opposed to is a rash war. What I am opposed to is the cynical attempt by Richard Perle and Paul Wolfowitz and other armchair, weekend warriors in this administration to shove their own ideological agendas down our throats, irrespective of the costs in lives lost and in hardships borne.
What I am opposed to is the attempt by political hacks like Karl Rove to distract us from a rise in the uninsured, a rise in the poverty rate, a drop in the median income — to distract us from corporate scandals and a stock market that has just gone through the worst month since the Great Depression. That’s what I’m opposed to. A dumb war. A rash war. A war based not on reason but on passion, not on principle but on politics. Now let me be clear — I suffer no illusions about Saddam Hussein. He is a brutal man. A ruthless man. A man who butchers his own people to secure his own power. He has repeatedly defied UN resolutions, thwarted UN inspection teams, developed chemical and biological weapons, and coveted nuclear capacity. He’s a bad guy. The world, and the Iraqi people, would be better off without him. (emphasis mine)
There is quite a contrast there, wouldn’t you say?
Check out this quote from an article in the New York Times about Glenn Greenwald when he shared his approach to “journalism”…
“I approach my journalism as a litigator,” he said. “People say things, you assume they are lying, and dig for documents to prove it.”
Jonathon Chait talked about this quote in an interesting article he wrote comparing Glenn Greenwald to Ralph Nadar…
That is a highly self-aware account. Of course, the job description of a litigator does not include being fair. You take a side, assume the other side is lying, and prosecute your side full tilt. It’s not your job to account for evidence that undermines your case — it’s your adversary’s job to point that out.
Glenn Greenwald has clearly taken a side, just like he did in 2002 when the Cheney/Bush administration began to “exact vengeance on the perpetrators” and Glenn was there cheering them along.""",
"""Each President marks his term with a leadership legacy that defines his administration. For Barack Obama, one of the hallmark characteristics of his time at the helm is unapologetic pursuit of policies that help the country. While this may seem like a given, under the circumstances, Obama’s tough stances on some issues has nonetheless sparked heavy resistance from the political right.
In order to set his own pace, however; the President has had to first clean up many of the lingering issues that predate his administration. In addition to the wars in Afghanistan and Iraq, Obama has faced a hornet’s nest of residual domestic policy that has also required strong leadership to rectify. As a result, the President has maintained an unwavering commitment to tackle tough issues – both here and abroad.
Real Issues At-Hand

Effective leadership requires proactive approaches to public policy, but it also relies on the ability to react quickly, in response to issues that arise. President Obama was thrown into the fire immediately upon securing the position, forcing him to reconcile very real issues facing the country. From domestic economic concerns to multiple foreign wars, the current administration has been elbow-deep in major policy reform since taking control of the executive branch of government.

To some; the President’s actions are off-putting, due to the decisive and unapologetic strategies he has implemented. But when held-up to the alternatives, it becomes clear that the President’s responses to some of this century’s most challenging realities have been tough, yet prudent.

Ineffective International Organizations

In addition to Iraq and Afghanistan, Obama’s administration has faced vexing foreign policy concerns elsewhere; including aggression and human rights atrocities in Syria and Russia. And while international support is put-forth by organizations like the United Nations, Obama’s policies have had to account for the fact that help from the UN and others does not always sway outcomes significantly. As a result, bold U.S. strategies and foreign policy decisions made by the Obama administration illustrate the tough stances the President has adopted to protect American interests.

U.S. Economy

The recent global recession and meltdown of important U.S. markets took its toll on the country and the world. Unemployment, home mortgage foreclosures and other important indicators illustrated never-before-seen economy turmoil that eventually reached every sector of the U.S. economy. Even so, the President understands the economic might at his disposal and operates accordingly on the world stage. Bold moves Obama made to correct the housing market downturn and Wall Street waffling show how the President’s tough policies have led to productive outcomes.

Public Support Drives Policy Decisions

While each President exhibits autonomy in his leadership role, policy outcomes are also influences by prevailing public opinion. In the case of health care reform, Obama took a tough position, which didn’t align with the beliefs of the political right. Tea Party protests and other displays of dissent followed, but the prevailing need for public health care overshadowed the misgivings of a few citizens clinging to the status quo. Though tough, the President’s initiatives would have been dead in the water, if not for the support of forward-thinking Americans lending their voices and activism to the collective cause.

Obama’s Personal Style

The President’s style has been characterized as “inflexible” and “unwavering”, so Obama has left his indelible mark on policy outcomes of the past 6 years. While ideological gains are a part of each presidential administration, Obama’s bold actions eclipse some of the cronyism seen in prior administrations. As a result, what some see as overly rigid or inflexible pursuits are actually a reflection of the President’s personal style. Whether from working within the Chicago political machine, or gleaned from years mobilizing support for grass roots issues, the current President is not afraid to adhere to his core beliefs.

Strong leadership cues from President Obama lead detractors to call him out for being too tough, at times. In reality, however; the President’s track record of decisive moves is simply a reflection of the issues he has faced and the prevailing public support for his policies.

Author:

Daphne Holmes contributed this guest post. She is a writer from www.ArrestRecords.com and you can reach her at daphneholmes9@gmail.com.""",
"""Fox News is hurting our country and the very people it relies on for viewership. Since the Affordable Care Act became law, the right-wing in this country has been trying to convince people not to get insurance, not to trust “Obamacare” and they have instilled an irrational fear of it in a large swath of the country.

This story is just one illustration of how Fox News and the misinformation they spew has real consequences.

Dean Angstadt fells trees for a living.

He’s a self-employed, self-sufficient logger who has cleared his own path for most of his 57 years, never expecting help from anyone. And even though he’d been uninsured since 2009, he especially wanted nothing to do with the Affordable Care Act.

“I don’t read what the Democrats have to say about it because I think they’re full of it,” he told his friend Bob Leinhauser, who suggested he sign up.

Before I get to the rest of the story, read this quote from a piece by Eric Wemple about Dean Angstadt’s experience…

Asked if Fox News had molded his view of Obamacare, Angstadt responded, “Yeah, yeah – they get people fired up. You know what, I really do have a different outlook on it.”

I went for many years without health insurance and can relate to Dean’s situation. My family has a history of heart problems, my dad and his brother both died of heart attacks in their early 40’s…so yeah, going without health insurance as I approached 40 years old was a bit scary. Thankfully, I took my current job at the university and began taking care of myself after I got insurance through my employer. More of Dean’s story…

In 2011, Angstadt had a pacemaker and defibrillator implanted to help his ailing heart pump more efficiently. Not long after, the almost 6-foot, 285-pound man’s man was back in the woods, doing the Paul Bunyan thing.

But last summer, his health worsened again. It was taking him 10 minutes to catch his breath after felling a tree. By fall, he was winded after traveling the 50 feet between his house and truck.

“I knew that I was really sick,” said the Boyertown resident. “I figured the doctors were going to have to operate, so I tried to work as long as I could to save money for the surgery. But it got to the point where I couldn’t work.”

A friend had been trying to convince Dean to sign up for insurance through the exchanges, but he kept resisting, falling back on the propaganda that he had been fed – the bad information that was having a real impact on this mans life.

From time to time, Leinhauser would urge Angstadt to buy a plan through the ACA marketplace. And each time, Angstadt refused.

“We argued about it for months,” Angstadt said. “I didn’t trust this Obamacare. One of the big reasons is it sounded too good to be true.”

[...]

Leinhauser went to Angstadt’s house, and in less than an hour, the duo had done the application. A day later, Angstadt signed up for the Highmark Blue Cross silver PPO plan and paid his first monthly premium: $26.11.

“All of a sudden, I’m getting notification from Highmark, and I got my card, and it was actually all legitimate,” he said. “I could have done backflips if I was in better shape.”

Angstadt’s plan kicked in on March 1. It was just in time. Surgery couldn’t be put off any longer. On March 31, Angstadt had life-saving valve-replacement surgery.

“I probably would have ended up falling over dead” without the surgery, Angstadt said. “Not only did it save my life, it’s going to give me a better quality of life.”

Dean Angstadt will live to chop another tree.

But Fox News needs to die a quick and painful death.

H/T to Steve Benen at The Maddowblog.""",
"""Just when you think Fox News has reached a new low, they dig even deeper.

This Bundy Ranch standoff between a deadbeat rancher and the Bureau of Land Management has made Fox News’ Sean Hannity go full anti-American as he pours gasoline on a fire that has been burning since 1993, when Cliven Bundy first decided that he wanted to sponge off American tax payers and unlike the honest, hard working ranchers around him, stop paying grazing fees.

When Cliven Bundy declared “But I don’t recognize the United States government as even existing”, I knew it would feed the racist, drooling morons with exactly what they needed to get their hate fired back up. It seems just a little strange to me that the Second Amendment fetishists would embrace someone who doesn’t even accept the federal Constitution, from which they pluck the 2nd Amendment – to the exclusion of all other amendments as well as the original language.

Chez Pazienza at The Daily Banter has an excellent piece up which poses several excellent questions about Fox News’ role in this mess. Go Chez…

What do you do when a media outlet refuses to behave responsibly? When it peddles outright lies? When it preaches a kind of cultural armageddon? When it, directly or indirectly, stokes the fires of armed insurrection without any regard whatsoever for what could be the disastrous impact?

I personally think the Fox News machine has ruined our country. They have succeeded in brainwashing millions of people into believing in an alternate reality. Several studies have been done that revealed this phenomenon. From a Fairleigh Dickinson University study…

The largest effect is that of Fox News: all else being equal, someone who watched only Fox News would be expected to answer just 1.04 domestic questions correctly — a figure which is significantly worse than if they had reported watching no media at all. (emphasis mine)

They feed on these uninformed people. And beyond that, they also convince them not to believe any other media…you know, that whole “liberal media” bullshit that all of us liberals laugh uncontrollably at whenever someone says it.

Back to Chez’s piece, where he cites another example of Fox News catering to militia-types who like to overcompensate with large guns…

Four years ago, Glenn Beck regaled his Fox News viewers day after day, week after week with paranoid fever dreams about the coming collapse of America and the need for some kind of resistance to the encroaching progressive threat. He picked specific targets for his outrage, people and institutions he claimed were surreptitiously trying to destroy the country; one of them was the Tides foundation, a relatively obscure organization based in San Francisco that Beck had singled out as being part of the secret liberal cabal.

In July of 2010, a guy named Byron Williams, wearing body armor and armed heavily, got into a shootout with the California Highway Patrol, injuring two officers. Under questioning he said that he was driving to San Francisco with the goal of “starting a revolution” by killing members of the Tides foundation and the ACLU. Why? Because he watched a lot Glenn Beck and heeded the host’s warnings that his beloved country was about to be taken away from him if somebody didn’t do something. So Byron Williams did.

And he wasn’t the only one.

I want to know where the responsible media is on this? Do journalists even attempt to police their own profession anymore or have they just given up and joined in the propaganda/money-making scheme that seems to be working for Fox News. Something has to change soon, the crazy just keeps getting crazier.""",
"""The Republican Party is full of crazies, idiots and bullies. They’re like a giant clown circus on drugs.

The State of the Union address typically brings out the crazy, grandstanding boneheads from the party but last night we got a taste of the bully wing.

From Talking Points Memo…

“Let me be clear to you, you ever do that to me again I’ll throw you off this fucking balcony,” Grimm told NY1 reporter Michael Scotto following an on-air interview in the Capitol Rotunda. “No, no, you’re not man enough, you’re not man enough. I’ll break you in half. Like a boy.”

This is your modern Republican Party, folks. Intimidation, threats of violence and corruption all rolled into one steaming pile of crap. More from TPM…

At the Caribbean Tropics nightclub, Grimm allegedly threatened another patron — the estranged husband of a woman Grimm had entered the club with. Grimm reportedly said the man “don’t know who he’s fucking with … I’ll fuckin’ make him disappear where nobody will find him.” That same night, (and we’re glossing over the part where Grimm allegedly said “I’m a fucking F.B.I. agent, ain’t nobody gonna threaten me.”) Grimm allegedly returned to the club with another FBI agent and several New York City police officers. He then refused to let club patrons and employees leave, and allegedly told everyone to “get up against the fucking wall” and gave an order: “all the white people get out of here.” (emphasis mine)

I’m sure glad we have an election later in the year. Funny, you never hear about Democratic Representatives doing this kind of crap. Republicans have the market cornered on crazy politicians.""",
"""I have to hang my head and admit that I am fascinated by pop culture and the crazy world that revolves around it. The excesses of young Hollywood and the music industry makes me sick to my stomach. I can’t help but think of the people in my poor town who are struggling to feed their kids and keep a shelter over their heads, while thugs like Justin Bieber blow more money in a single party than many make in decade.

Waking up to the news that Justin Bieber, the Teflon Teen™, was finally busted for something, after getting away with so much shit over the years, made me smile.

When I heard that he resisted arrest, wouldn’t take his hands out of his pockets according to one report, I immediately thought of Roy Middleton, the Florida man who was shot at 15 times by police in his own driveway while retrieving some cigarettes out of the car. He was black, you know? Thankfully, he lived through it.

As the day goes on, the sensationalist cable new networks will use the Bieber story as a good reason to ditch all the other important stories happening in the world today. For example, our awesome President Barack Obama announced an initiative to combat rape yesterday and the evidence against Chris Christie and his bullying ways are growing in scope.

Based on the look on Justin’s face, he probably thinks it’s cool to get busted. He knows damn well that he can afford the best lawyers you can buy. Something tells me this isn’t the last mug shot we will see from the Teflon Teen™.

(I don’t really have the trademark for Teflon Teen, so feel free to use the hell out of it. :)""",
"""What is it about Chris Christie that so enthralls some in the media? I have to wonder if it is because he screams at his constituents. Mother Jones posted an article with 8 videos of Christie berating and yelling at people, go take a look to see what “straight-talk” is according to his adoring friends in the media.

The Star-Ledger had an opinion piece that makes fun of the people I am now referring to as the “Benghazi stupid.” This group of knuckle dragging idiots makes up the leadership of the Republican party these days, along with the reactionary, hateful nihilists that are easily distracted by the word “Benghazi.”

From the Star-Ledger piece…

Somehow, the right’s response to Chris Christie’s still-breaking Bridgegate scandal has devolved into this: Why are you writing about New Jersey traffic jams, because Benghazi!

In letters to newspapers and online comments, in phone calls to their favorite conservative radio and TV pundits, conservatives are in a state of collective denial: They refuse to acknowledge there’s anything to Gov. Chris Christie and the George Washington Bridge scandal until President Obama and the consular attack in Benghazi get equal time.

[...]

What’s the difference? Here are three reasons the unrelated Bridgegate and Benghazi stories aren’t getting equal time – and shouldn’t.

Intent: America’s press corps has looked at Benghazi, the IRS scandal and the other Obama-related scandals tossed around last weekend. In each case, the facts dampened the early cries of conspiracy and cover-up. In Benghazi, neither congressional investigators nor the New York Times found evidence to support the idea of a concerted executive branch failure or cover-up. In the IRS fiasco, an investigation found both conservative and liberal political groups were subject to review – and everyone got what they wanted, anyway.

Coverage: It’s hard to argue that Benghazi, the IRS scandal or Obamacare’s glitchy website weren’t covered in full. Each story was subject to intense coverage when it broke – just as Bridgegate is breaking now. To expect coverage of old stories to increase because of an uncomfortable new story is silly.

Just as silly are suggestions that New Jersey media, including The Star-Ledger, are spending too much time covering a breaking story of corruption in the governor’s office. Because Benghazi?

Cover-ups: Each scandal resuscitated by the right last week began with cover-up allegations that have faded under the bright lights of media coverage and federal investigation.

Meanwhile, new evidence that Christie’s aides tried to cover their tracks is surfacing as thousands of newly released documents and e-mails are made public. None of the evidence suggests the governor was involved at that level, but there are a lot of questions about the GWB lane closures that still haven’t been answered.

I watched Chris Christie’s marathon press conference the other day and he laid it on pretty thick for his adoring media audience. For those of us who aren’t swayed by his tactics, it didn’t pass the smell test. And now, with the release of more information about who in his office knew what was going on, it stinks even more.

He had to know what was going on."""
]

In [19]:
krugman_posts = ["""Last night Atif Mian and I flew up to Boston for a conference — and as I slid into my seat, who should I see staring at me but Ron Paul. It turned out that all of the seatback screens in the plane were showing Newsmax TV — who knew there was such a thing? Is it there to serve people who find Fox News too liberal? — and as best I could tell from the visual context (the sound was blessedly off), the elder Paul was lecturing us about monetary policy.

This sort of thing is obviously an important part of the reason we’re living in an age of derp. Events and data may have made nonsense of claims that the Fed’s policies would inevitably produce runaway inflation, and made those insisting on such claims look like fools; but there’s a large audience of people who, pulled in by affinity fraud, live in a bubble where they never hear about such evidence.

Truly, we live in a world in which people feel entitled not just to their own opinions but their own facts.""",
"""I gather that some readers were puzzled by my use of the term “derp” with regard to peddlers of inflation paranoia, even though I’ve used it quite a lot. So maybe it’s time to revisit the concept; among other things, once you understand the problem of derpitude, you understand why I write the way I do (and why the Asnesses of this world whine so much.)

Josh Barro brought derp into economic discussion, and many of us immediately realized that this was a term we’d been needing all along. As Noah Smith explained, what it means — at least in this context — is a determined belief in some economic doctrine that is completely unmovable by evidence. And there’s a lot of that going around.

The inflation controversy is a prime example. If you came into the global financial crisis believing that a large expansion of the Federal Reserve’s balance sheet must lead to terrible inflation, what you have in fact encountered is this:
I’ve indicated the date of the debasement letter for reference.

So how do you respond? We all get things wrong, and if we’re not engaged in derp, we learn from the experience. But if you’re doing derp, you insist that you were right, and continue to fulminate against money-printing exactly as you did before.

The same thing happens when we try to discuss the effects of tax cuts — belief in their magical efficacy is utterly insensitive to evidence and experience.

Now, not every wrong idea — or claim that I disagree with — is derp. I was pretty unhappy with the claim that doom looms whenever debt crosses 90 percent of GDP, and not too happy with the later claims that the relevant economists never said such a thing; that’s what everyone from Paul Ryan to Olli Rehn heard, and they were not warned off. But there has not, thankfully, been a movement insisting that growth does too fall off a cliff at 90 percent, so this is not a derp thing.

But there is, as I said, a lot of derp out there. And what that means, in turn, is that you shouldn’t pretend that we’re having a real discussion when we aren’t. In fact, it’s intellectually dishonest and a public disservice to pretend that such a discussion is taking place. We can and indeed are having a serious discussion about the effects of quantitative easing, but people like Paul Ryan and Cliff Asness are not part of that discussion, because no evidence could ever change their view. It’s not economics, it’s just derp.

Now, saying this brings howls of rage, accusations of rudeness and being nasty. But what else can one do?""",
"""David Wessel has a very nice explainer in the WSJ — although I wonder how the editor allowed his citation of a particular expert under point #2 to slip through. One thing he doesn’t do, however, is make it clear that zero is not a magic red line here — as even the IMF has made a point of emphasizing, too-low inflation has all the adverse effects of outright deflation, just to a lesser degree.

Most notably, the euro area currently has 0.8 percent core inflation, far below its 2 percent target, which is itself too low. This means that Europe is already in a lowflationary trap, qualitatively the same as a deflationary trap.""",
"""At this point in the great inflation-deflation debate, a lot of what the inflationistas have to say takes the form of whining about the rudeness of their critics — of course, me in particular. I would say that this is a de facto confession that they’ve run out of substantive defenses for their position — although I guess I would say that, wouldn’t I?

But there’s something else you should know: the inflation derpers aren’t just ignorant about monetary policy, they also don’t understand the rules of argument. In particular, the constant complaint about “ad hominem” attacks shows that they don’t know what that means.

I think the Wikipedia definition is pretty good: an ad hominem is

a form of criticism directed at something about the person one is criticizing, rather than something (potentially, at least) independent of that person.

So if, for example, somebody discussing my views on monetary policy refers to me as “Enron consultant Paul Krugman”, that’s ad hominem. But if I say that inflationistas have been

bobbing and weaving, refusing to acknowledge having said what they said, being completely unwilling to admit mistakes.

that’s really not ad hominem; I’m attacking how these people argue, not their personal attributes.

What about the lexicon we’ve developed over the course of the past few years — zombies, cockroaches, confidence fairies, derp? These are all terms directed at arguments, not people; no, I didn’t call Olli Rehn a cockroach, just his historically ignorant assertion that Keynes wouldn’t have called for fiscal stimulus in the face of high debt.

The point is that at no point, as far as I know, have I relied on personal attacks as a substitute for substantive argument. I never accuse someone of practicing derp without showing that he is, indeed, practicing derp.

Still, why use such colorful language? To get peoples’ attention, of course, and to highlight the sheer scale of the folly. And it’s working, isn’t it?

Now, the people who make zombie arguments and engage in derp feel deeply insulted by all of this. But if you’re going to engage in public debate, with very real policy concerns that affect the lives of millions at stake, you are not entitled to have your arguments treated with respect unless they deserve respect.

One more thing: I also don’t think that the derp brigade understands what it means to argue from authority. When I say that you shouldn’t opine on monetary policy unless you’re willing to invest some time on understanding the monetary debate, I am saying exactly that. I’m not saying that you need a Ph.D. or a chair at a fancy university; I’m saying that you need to do your homework.

In a better world, none of this would be relevant. Policy disputes would be based on defensible, well-informed positions, on which reasonable people could disagree, and people who were proved wrong would acknowledge that fact and revise their views. Also, everyone would get a pony.""",
"""Can there be too much Lucius? Not to my taste. And the band does want people to know that they have released a special enhanced version of their album, with a bunch of bonus tracks including live performances (which is where they really shine). And here’s something unusual: a live performance video assembled from videos taken from many people who were there. If this doesn’t make you smile, I feel sorry for you:""",
"""Via Business Insider, Zero Hedge directed its readers to an “excellent interview” in which Jim Rogers declared that “we are all going to pay a terrible price for all this money-printing and debt.” And I asked the obvious question: How long has Rogers been predicting a printing-press-and-deficits disaster?

The answer is, a very, very long time. Here he is in October 2008 — six full years ago — declaring that we were setting the stage for a “massive inflation holocaust.”

Now, you might have thought that after years of being completely wrong (with a diversion into inflation trutherism), one of two things would happen: 1. Rogers would question his own premises 2. People would stop taking his views on macroeconomics seriously.

But no. His views haven’t changed (and given what we’ve seen from others of similar views, he would deny that anything was amiss with his predictions); and he’s still treated by financial media as a source of deep wisdom.

The ability of inflation derp to persist, even flourish, in an age of disinflation remains remarkable.""",
"""From the beginning, economists who had studied the Great Depression warned that policy makers needed, above all, to be careful not to pull another 1937 — a reference to the fateful year when FDR prematurely tried to balance the budget and the Fed prematurely tried to normalize monetary policy, aborting the recovery of the previous four years and sending the economy on another big downward slope.

Unfortunately, these warnings were ignored. True, the Fed at least stood up to the inflationistas demanding tighter money now now now; but its actions were at the least hobbled by the chorus. The ECB actually did raise rates for a while, as did the Riksbank in Sweden. And fiscal austerity driven by fear of the invisible bond vigilantes (and justified by faith in the confidence fairy) was the norm everywhere, although worse in Europe.

And now things are sliding everywhere. Actually, Europe already had one 1937, with its slide into a double-dip recession; but now it’s very much looking like another. And the world economy as a whole is weakening fast.

So now we have another milestone: Earlier today the 10-year yield dropped below 2 percent. It’s up again slightly as I write this, but all the market signals are saying that once again the big risk is deflation or at least very sub-par inflation.

I hope that the Fed will stop talking about exit strategies for a while. We are by no means out of the Lesser Depression.""",
"""Cliff Asness, one of the signers of the infamous open letter warning Ben Bernanke that his policies risked debasing the dollar, weighs in with a complaint that I am being a big meanie. As Brad DeLong immediately notes, what Asness mainly ends up doing is showing that he doesn’t at all get the whole notion of the liquidity trap, and the resulting irrelevance of monetary expansion to both prices and output.

Clearly, Asness has never read anything at all on the subject — not what I’ve written, not what Mike Woodford has written, not what Ben Bernanke has written. And he seems to view the failure of inflation to follow from quantitative easing as some sort of weird coincidence, not what anyone who applied basic macroeconomics to the situation predicted.

Now, I understand that busy people can’t keep track of everything, and even that you can sometimes be a successful money manager without reading up on monetary economics. But if you’re one of those people who don’t have time to understand the monetary debate, I have a simple piece of advice: Don’t lecture the chairman of the Fed on monetary policy.""",
"""Olivier Blanchard has gotten a lot of ribbing, from me among others, for his 2008 paper proclaiming that “the state of macro is good.” My critique was that Olivier was in a state of denial about the Dark Age of macroeconomics; when crisis struck and action became necessary, it became all too clear that freshwater macro had unlearned everything Keynes and Hicks had taught – and also that the desperate New Keynesian attempt to appease the rational expectations crowd had not only failed in that purpose, but arguably hobbled efforts to think clearly about anything that didn’t fit easily into a model where everything except price stickiness reflected maximization..

I would argue that Olivier’s latest version, which concedes that there are “dark corners” where the rational expectations approach doesn’t work, is still trying too hard to appease the unappeasable. But Arnold Kling offers a different critique: he thinks that Blanchard is demonstrating “modeling hubris.” And that, I’d argue, is all wrong.

First of all, whenever somebody claims to have a deeper understanding of economics (or actually anything) that transcends the insights of simple models, my reaction is that this is self-delusion. Any time you make any kind of causal statement about economics, you are at least implicitly using a model of how the economy works. And when you refuse to be explicit about that model, you almost always end up – whether you know it or not – de facto using models that are much more simplistic than the crossing curves or whatever your intellectual opponents are using.

Think, in particular, of all the Austrians declaring that the economy is too complicated for any simple model – and then confidently declaring that the Fed’s monetary expansion would cause runaway inflation. Whatever they may have imagined, they were in practice using a crude quantity-theory model of the price level.

And as I have often tried to explain, the experience of the past six years has actually been a great vindication for those who relied on a simple but explicit model, Hicksian IS-LM, which made predictions very much at odds with what a lot of people who didn’t use explicit models were sure would happen.

Suppose that you didn’t know about IS-LM and the concept of the liquidity trap. You would (and many did) look at the growth of the monetary base, and predict huge inflation:
And you could (and many did) look at government borrowing, and predict soaring interest rates:
But if you understood IS-LM, you realized that both the relationship between money and inflation and the relationship between borrowing and interest rates break down at the zero lower bound; and so they did.

If you don’t think these successful predictions are a big deal, go back and read the dismissive, vituperative comments those of us who predicted low inflation and interest rates faced back in 2009.

And a somewhat related point: when people claim to have a sophisticated understanding that transcends models, what, exactly, would they ever regard as evidence that their sophisticated understanding is, you know, wrong?""",
"""I’m late coming in on the Tirole Nobel – busy with real life – and many people have already weighed in on his contribution. But I though I might still have something useful to say about what the New Industrial Organization, of which he was the most important figure, actually did – namely, it made it safe to be strategically silly, to the great benefit of economics.

What do I mean by that? Before the new IO, economists wrote about perfect competition and monopoly, then acknowledged (if they were honest) that most of the real economy seemed to consist of oligopoly – competition among the few – but did little there except some hand-waving. Why? Because there was no general model of oligopoly.

And there still isn’t. When you have a small number of players, each able to have a significant effect on prices, lots of things can happen. They can collude – maybe implicitly, if there is an effectively enforced antitrust law; but what are the limits of collusion, and why and when does it sometimes break down? We like to assume that firms maximize profits, but what does that even mean when there are small-group interactions that create prisoners’-dilemma-type situations?

And yet you do want to model the economy, to think about stuff – and sometimes that stuff can’t be modeled without addressing imperfect competition. That was very much the case in my home field of trade, where even trying to model the role of increasing returns meant dealing with the fact that increasing returns internal to firms must cause perfect competition to break down.

Before the new IO came along, the way economics dealt with such issues was to assume them away. Increasing returns as a cause of trade? Hey, you can’t deal with that because we don’t have a theory of imperfect competition, so we have to assume that it’s all comparative advantage. (Harry Johnson once wrote a more or less triumphant paper to that effect.) Investment in R&D, and the temporary market power that results, as a source of technological progress? No can do.

What new IO brought was not so much a solution as an attitude. No, we don’t have a general model of oligopoly – but why not tell some stories and see where they lead? We can simply assume noncooperative price (or quantity) setting; yes, real firms are probably going to find ways to collude, but we might learn interesting things by working through the case where they don’t. We can make absurd assumptions about tastes and technology that lead to a tractable version of monopolistic competition; no, real markets don’t look like that, but why not use this funny version to think about increasing returns in trade and growth?

Basically, the new IO made it OK to tell stories rather than proving theorems, and thereby made it possible to talk about and model issues that had been ruled out by the limits of perfect competition. It was, I can tell you from experience, profoundly liberating.

Of course, there came a later phase when things were too liberated – when a smart grad student could produce a model to justify anything. Time for empirical work! But by then a lot had been achieved."""]

In [20]:
friedman_posts = ["""This conference is based on one premise: What would we be talking about as a country if we were talking about what’s really important?

What we’d be talking about, of course, is one of the biggest things on the lips of everyone you talk to today: How is my kid going to get a job, and how is my middle-aged mother or father or brother or sister going to keep theirs? As I go around America — indeed, as I go around the world — this is among the biggest questions out there today. Everyone senses that something is up — something has changed in the world of technology that is changing every job, every workplace, every institution — but no one is explaining it to people, let alone giving them the guidance they need for how to navigate this new era. That is what we hope to do today.

And by the way, what people are sensing is right. Data released the other day by the Federal Reserve revealed that 44 percent of young college graduates in 2012 were working in jobs that didn’t require a college degree, up from 38 percent before the recession in 2007. Think about that: Nearly half the people who have either paid out or gone into debt to cover tuitions of $10,000, $50,000 or $250,000 to get a college degree are not working in a job that requires it.

Houston, we have a problem.

No doubt, a big part of the explanation is traceable to the sharp drop-off in demand for labor that accompanied the 2008 Great Recession. But I would argue that something else is going on and it is this: The Great Recession coincided with a Great Inflection – a huge technological shift of Gutenberg-scale proportions in how the world is wired, in what machines can now do in terms of substituting for human labor and in how many people around the world can compete for any job. And this technological shift is roiling the job market.

For starters, it is wiping out what sustained the American middle class for the six decades after World War II – the high-wage, middle-skilled job. There is no such thing anymore. There is only a high-wage, high-skilled job. Stefanie Sanford from the College Board is here today and will talk about that.

In addition, every middle class job is being pulled in three directions at once. It is being pulled up, out and down – all at the same time. How so? The New York Times carried a story the other day about dairy farms in upstate New York which are now using robots to milk cows. Another of our speakers here, Andy McAafee, co-author of “The Second Machine Age,” will tell you that George Jetson drove his own flying saucer to work. In other words the sci-fi creators of the Jetsons could not imagine 50 years ago what is right out our door today — self-driving cars from Google, or in George Jetson’s case, a self-driving flying saucer.

It is for all of these reasons that our friend Tony Wagner, the Harvard education specialist, keeps warning us that it is not enough any more to graduate high school college ready, you will have to graduate “innovation ready.” The kind of jobs you will want to invent for yourself will also have to be different from my generation. The historian Walter Russell Mead noted recently:

“In the 19th century most Americans spent their time working with animals and plants outdoors in the country. In the 20th century most Americans spent their time pushing paper in offices or bashing widgets in factories. In the 21st century most of us are going to work with people, providing services that enhance each others’ lives… We are going to have to discover the inherent dignity of work that is people to people rather than people to things. We are going to have to realize that engaging with other people, understanding their hopes and their needs, and using our own skills, knowledge and talent to give them what they want at a price they can afford is honest work.”

These are but a few of the challenges we face, but it’s always been my view that the best way to think about how to address these challenge is by looking at the whole supply chain — what are the big technology shifts, what new industries are they spawning, what new jobs are they creating, what new skills are required to succeed in these jobs, what does that mean for post secondary education, what does that mean for K-12 education and therefore how should we spend our tax dollars preparing our communities for this world.

We know something big is happening technologically. But what is it exactly? We know the answer is more and better education. But what does that look like in a classroom? We know it will require each of us to learn and relearn more often. But how? Am I supposed to go back to college every four years? We know that our kids are likely to work in many more jobs than we ever had to, but how does one go about applying and interviewing for a job that was just invented last week?

I think before the day is over many of you may come to the same conclusion I have. We need a whole new social contract, a whole public private partnership for this new world, because more will be on you. The way I like to put it is that we are now in a 401(k) world. We have gone from a world of “defined benefits” — just do your job every day, show up, be average and here are the benefits you will get — into a world of “defined contributions” — your wages and returns will now be more and more directly correlated to you exact contribution, and with big data we will get better and better at measuring just that every day.

If you turn on late-night TV or look at the subway ads, there are lot of people who will tell you how to invest your 401(k). But there are very few who will tell you how to invest in yourself — in a world where more will be on you. Governments role is more and more going to be to catch those less fortunate who are left behind but also to empower everyone to invest in themselves and in lifelong learning.

So these are the questions we are going to address today, because we desperately need our next election to be about the mix of policies needed to address these new challenges – and I stress the word mix — over and above the simplistic solutions of how to make rich people poorer by taxing them more or just pretending that cutting taxes and shrinking government will make poor people rich. We have here today the very best people I know of in the world to inform this discussion. Our goal, to use the language of one of speakers this morning Dov Seidman, is not just to shift the debate, our goal is to elevate it — otherwise we can’t elevate the country.""",
"""Reading the papers these days I find that the two world leaders who stir the most passion in me are Pope Francis and Vladimir Putin, the president of Russia. One is everything you’d want in a leader, the other everything you wouldn’t want. One holds sway over 1.2 billion Roman Catholics, the other over nine time zones. One keeps surprising us with his capacity for empathy, the other by how much he has become a first-class jerk and thug. But neither can be ignored and both have an outsized influence on the world today.

First, the pope. At a time when so many leaders around the world are looking to promote their political fortunes by exploiting grievances and fault lines, we have a pope asking his flock to do something hard, something outside their comfort zone, pushing them to be more inclusive of gays and divorced people.

Yes, Francis was rebuffed by conservative bishops at a recent Vatican synod when he asked them to embrace the notion that “homosexuals have gifts and qualities to offer to the Christian community,” adding, “are we capable of welcoming these people, guaranteeing to them a fraternal space in our communities?”

But, as an editorial in this paper noted: “The very fact that Francis ordered church leaders to address these challenges seems a landmark in Vatican history.” The pope asked that rejected language be published for all to see, while also cautioning against “hostile inflexibility — that is, wanting to close oneself within the written word, and not allowing oneself to be surprised by God.”

“Hostile inflexibility?” Whose leadership does that describe? Look at Putin’s recent behavior: His military was indirectly involved in downing a Malaysian airliner over Ukraine and his K.G.B. has not only been trying to take a bite out of Ukraine but is nibbling on Estonia, Georgia and Moldova, all under the guise of protecting “Russian speakers.”

I opposed NATO expansion because I believed that there are few global problems that we can solve without the help of Russia. By expanding NATO at the end of the Cold War, when Russia was weak, we helped to cultivate a politics there that would one day be very receptive to Putin’s message that the West is ganging up on Russia. But, that said, the message is a lie. The West has no intention of bringing Ukraine into NATO. And please raise your hand if you think the European Union plans to invade Russia.

Yet Putin just exploits these fears for two reasons. First, he has a huge chip on his shoulder — no, excuse me; he has a whole lumberyard there — of resentment that Russia is no longer the global power it once was. But rather than make Russia great again by tapping its creative people — empowering them with education, the rule of law and consensual politics to realize their full potential — he has opted for the shortcut of tapping his oil and gas wells and seizing power from his people.

And instead of creating a Russia that is an example to its neighbors, he relies on the brute force that his oil and gas can still buy him. While he rails against NATO, he is really afraid of European Union expansion — that Ukrainians would rather embrace the E.U. market and democracy rules than their historical ties to Russia because they know that through the E.U. they can realize potentials that would never be possible with Russia.

By seizing Crimea and stoking up nationalism, Putin was not protecting Russia from NATO. He was protecting himself from the viruses of E.U. accountability and transparency, which, if they took hold in Ukraine, could spread to Moscow, undermining his kleptocracy.

Normally, I wouldn’t care, but when the world is dividing between zones of order and disorder, and the world of order needs to be collaborating to stem and reverse disorder, the fact that Putin is stoking disorder on Russia’s borders, and not collaborating to promote order in the Middle East, is a real problem. What’s more worrying is that the country he threatens most is Russia. If things go bad there — and its economy is already sagging under Western sanctions — the world of disorder will get a lot bigger.

That is why Putin’s leadership matters, and so does the pope’s. I’m focused on Putin because I think he is making the world a worse place for bad reasons, when he could make a difference in Europe and the Middle East with just an ounce more decency and collaboration. America, too, has plenty to learn from the pope’s humility, but say what you will, we’re still focused on trying to strengthen the global commons, whether by protecting people from jihadists in Iraq or fighting Ebola in Africa. We could do more. Putin needs to do a lot more.

“The best leaders don’t set timid and selfish goals that are easy to meet but instead set bold and inclusive goals that are hard to achieve,” remarked Timothy Shriver, the chairman of the Special Olympics, who has just written a book on leadership, “Fully Alive: Discovering What Matters Most.” “We’re all looking for ways to make sense of a world without a center, but we’ll only find that in people who lead with authentic humility and reckless generosity.""",
"""Is it just my imagination or is there a global oil war underway pitting the United States and Saudi Arabia on one side against Russia and Iran on the other? One can’t say for sure whether the American-Saudi oil alliance is deliberate or a coincidence of interests, but, if it is explicit, then clearly we’re trying to do to President Vladimir Putin of Russia and Iran’s supreme leader, Ayatollah Ali Khamenei, exactly what the Americans and Saudis did to the last leaders of the Soviet Union: pump them to death — bankrupt them by bringing down the price of oil to levels below what both Moscow and Tehran need to finance their budgets.

Think about this: four oil producers — Libya, Iraq, Nigeria and Syria — are in turmoil today, and Iran is hobbled by sanctions. Ten years ago, such news would have sent oil prices soaring. But today, the opposite is happening. Global crude oil prices have been falling for weeks, now resting around $88 — after a long stretch at $105 to $110 a barrel.

The price drop is the result of economic slowdowns in Europe and China, combined with the United States becoming one of the world’s biggest oil producers — thanks to new technologies enabling the extraction of large amounts of “tight oil” from shale — combined with America starting to make exceptions and allowing some of its newfound oil products to be exported, combined with Saudi Arabia refusing to cut back its production to keep prices higher, but choosing instead to maintain its market share against other OPEC producers. The net result has been to make life difficult for Russia and Iran, at a time when Saudi Arabia and America are confronting both of them in a proxy war in Syria. This is business, but it also has the feel of war by other means: oil.

The Russians have noticed. How could they not? They’ve seen this play before. The Russian newspaper Pravda published an article on April 3 with the headline, “Obama Wants Saudi Arabia to Destroy Russian Economy.” It said: “There is a precedent [for] such joint action that caused the collapse of the U.S.S.R. In 1985, the Kingdom dramatically increased oil production from 2 million to 10 million barrels per day, dropping the price from $32 to $10 per barrel. [The] U.S.S.R. began selling some batches at an even lower price, about $6 per barrel. Saudi Arabia [did not lose] anything, because when prices fell by 3.5 times [Saudi] production increased fivefold. The planned economy of the Soviet Union was not able to cope with falling export revenues, and this was one of the reasons for the collapse of the U.S.S.R.”

Indeed, the late Yegor Gaidar, who between 1991 and 1994 was Russia’s acting prime minister, observed in a Nov. 13, 2006, speech that: “The timeline of the collapse of the Soviet Union can be traced to Sept. 13, 1985. On this date, Sheikh Ahmed Zaki Yamani, the minister of oil of Saudi Arabia, declared that the monarchy had decided to alter its oil policy radically. The Saudis stopped protecting oil prices. ... During the next six months, oil production in Saudi Arabia increased fourfold, while oil prices collapsed. ... The Soviet Union lost approximately $20 billion per year, money without which the country simply could not survive.”

Neither Moscow nor Tehran will collapse tomorrow. And if oil prices fall below $70 you will see a drop in U.S. production, as some exploration won’t be cost effective, and prices could firm up. But have no doubt, this price falloff serves U.S. and Saudi strategic interests and it harms Russia and Iran. Oil export revenues account for about 60 percent of Iran’s government revenues and more than half of Russia’s.

The price decline is no accident. In an Oct. 3 article in The Times, Stanley Reed noted that the sharp drop in oil prices “was seen as a response to Saudi Arabia’s signaling ... to the markets that it was more interested in maintaining market share than in defending prices. Saudi Aramco, the national oil company, stunned markets by announcing that it was cutting prices by about $1 a barrel to Asia, the crucial growth market for the Persian Gulf producers, as well as by 40 cents a barrel to the United States.” The Times also noted that with America now producing so much more oil and gas, “net oil imports to the United States have fallen since 2007 by 8.7 million barrels a day, ‘roughly equivalent to total Saudi and Nigerian exports,’ according to a recent Citigroup report.”

This resource abundance comes at a time when we’ve also hit a “gusher” of energy technology in Silicon Valley, which is supplying us with unprecedented gains in energy efficiency and productivity, savings that may become as impactful as shale in determining our energy security and global strength. Google, through Nest, and Apple through coding in the iPhone software, are making it easier for average Americans to manage and save energy at home or work.

Bottom line: The trend line for petro-dictators is not so good. America today has a growing advantage in what the former Assistant Energy Secretary Andy Karsner calls “the three big C’s: code, crude and capital.” If only we could do tax reform, and replace payroll and corporate taxes with a carbon tax, we’d have a formula for resiliency and success far better than any of our adversaries.""",
"""AN Iraqi official recently told me this story: When the Islamic State, also known as ISIS, took over Mosul in the summer, the Sunni jihadist fighters in ISIS, many of whom were foreigners, went house to house. On the homes of Christians they marked “Nassarah,” an archaic Arabic term for Christians. But on the homes of Shiites they marked “Rafidha,” which means “those who reject” the Sunni line of authority as to who should be caliph, or leader of the Muslim community, after the death of the Prophet Muhammad. But here’s what was interesting, the Iraqi official said, the term “Rafidha” was largely unknown in Iraq to describe Shiites. It is a term used by Wahhabi fundamentalists in Saudi Arabia. “We did not know this word,” he told me. “This is not an Iraqi term.”

I was intrigued by this story because it highlighted the degree to which ISIS operates just like an “invasive species” in the world of plants and animals. It is not native to either the Iraqi or Syrian ecosystems. It never before grew in their landscapes.

I find it useful at times to use the natural world to illuminate trends in geopolitics and globalization, and this is one of them. The United States National Arboretum website notes that “invasive plant species thrive where the continuity of a natural ecosystem is breached and are abundant on disturbed sites like construction areas and road cuts. ... In some situations these nonnative species cause serious ecological disturbances. In the worst cases, invasive plants ... ruthlessly choke out other plant life. This puts extreme pressure on native plants and animals, and threatened species may succumb to this pressure. Ultimately, invasive plants alter habitats and reduce biodiversity.”

I can’t think of a better way to understand ISIS. It is a coalition. One part consists of Sunni Muslim jihadist fighters from all over the world: Chechnya, Libya, Britain, France, Australia and especially Saudi Arabia. They spread so far, so fast, despite their relatively small numbers, because the disturbed Iraqi and Syrian societies enabled these foreign jihadists to forge alliances with secular, native-born, Iraqi and Syrian Sunni tribesmen and former Baathist army officers, whose grievances were less religious and more about how Iraq and Syria were governed.

Today, ISIS — the foreigners and locals together — is putting pressure on all of Iraq’s and Syria’s native species with the avowed goal of reducing the diversity of these once polycultural societies and turning them into bleak, dark, jihadist, Sunni fundamentalist monocultures.

It is easy to see how ISIS spread. Think about the life of a 50-year-old Iraqi Sunni male from Mosul. He first got drafted to fight in the eight-year Iran-Iraq war that ended in 1988. Then he had to fight in the Persian Gulf war I after Saddam Hussein invaded Kuwait. Then he lived under a decade of U.N. sanctions that broke Iraq’s middle class. Then he had to endure the years of chaos that followed the U.S. invasion, which ended with a corrupt, brutal, pro-Iranian Shiite regime in Baghdad led by Nuri Kamal al-Maliki that did all it could to keep Sunnis poor and powerless. This was the fractured political ecosystem in which ISIS found fertile ground.

How do you deal with an invasive species? The National Arboretum says you should “use systemic herbicides carefully” (President Obama’s air war), while also constantly working to strengthen and “preserve healthy native plant habitats” (Obama’s effort to forge a national unity government in Baghdad with Shiites, Sunnis and Kurds together).

Generally speaking, though, over the years in Iraq and Afghanistan we have overspent on herbicides (guns and training) and underinvested in the best bulwark against invasive species (noncorrupt, just governance). We should be pressing the Iraqi government, which is rich with cash, to focus on delivering to every Iraqi still under its control 24 hours of electricity a day, a job, better schools, more personal security and a sense that no matter what sect they’re from the game is not rigged against them and their voice will count. That is how you strengthen an ecosystem against invasive species.

“It was misgovernance which drove Iraqis to contemplate a relationship with ISIS with the view that it was less detrimental to their interests than their own (Shiite-led) government,” explained Sarah Chayes, a senior associate at the Carnegie Endowment who is a former U.S. adviser in Afghanistan and author of the upcoming “Thieves of State: Why Corruption Threatens Global Security.” The Iraqi Army we built was seen by many Iraqi Sunnis “as the enforcer of a kleptocratic network.” That army got “sucked dry by the cronies of Maliki so it became a hollow shell that couldn’t withstand the first bullet.”

The goal of ISIS now is to draw us in, get us to bomb Sunni towns and drive the non-ISIS Sunnis away from America and closer to ISIS, “because,” notes Chayes, “ISIS knows it can’t survive without the support of these non-ISIS Sunnis.”

We always overestimate military training and force and underestimate what Arabs and Afghans want most: decent and just governance. Without the latter, there is no way to cultivate real citizens with a will to fight — and without will there is no training that matters.

Ask any general — or gardener.""",
"""I’m sure there are many technical explanations for the recent breakdowns in Secret Service protection that allowed an armed intruder to run right through the front door of the White House and an armed felon to ride on an elevator with President Obama. But I’d also put some blame on the nation’s political class.

Just look at Washington these days and listen to what politicians are saying and watch how they spend their time. You can’t help but ask: Do these people care a whit about the country anymore? Is there anybody here on a quest for excellence, for making America great?

Yes, yes, I know. They’re all here to do “public service.” But that is not what it looks like. It actually looks as if they came to Washington to get elected so they could raise more money to get re-elected. That is, until they don’t get re-elected. Then, like the former House majority leader, Eric Cantor, they can raise even more money by cashing in their time on Capitol Hill for a job and a multimillion-dollar payday from a Wall Street investment bank they used to regulate.

Getting elected and raising money to get re-elected — instead of governing and compromising in the national interest — seems to be all that too many of our national politicians are interested in anymore. There are exceptions, to be sure, but it feels as if many do not take pride in their work in government.

We’re at war in the Middle East, with American military lives on the line, but Congress could not stir itself to return from a pre-election recess to either debate the wisdom of this war or give the president proper legal authorization, let alone take some responsibility. When everyone is so busy running, is it any surprise that no one is running the federal government?

According to PolitiFact, “Wyoming Republican Senator John Barrasso said, ‘This is the earliest Congress has adjourned in over 50 years.’ ... Virginia Democratic Senator Tim Kaine called it ‘the second-earliest recess before a midterm since 1960.’ Both senators are correct (if you excuse Barrasso’s use of adjourn instead of recess). For Barrasso, ‘over 50 years,’ takes us back to any year up to 1963, or 51 years ago. The record shows that since 1963, Congress often has taken short breaks in September, given lawmakers several weeks in October to campaign in election years, and even closed the books for the year in October. But an extended break from mid-September to mid-November has not occurred. Kaine’s claim is spot on.”

What does this have to do with the Secret Service lapses? It certainly doesn’t excuse them, but if you’re a federal worker today and you look up at the “adults” who are supposed to be supervising you, what do you see? You see too many self-interested, self-indulgent politicians who are only there to grandstand, spend most of their time raising money to win elections and then, when you, as a federal worker, make a mistake, be the first to rush to the microphones with feigned concern to investigate your competence — as long as the cameras are running.

Tell me that doesn’t filter down to every department, including the Secret Service. When so many above you are just cynically out for themselves, it saps morale, focus and discipline. If so many above you are just getting theirs, well then, why shouldn’t Secret Service agents doing advance work for the president’s trip to Colombia in April 2012 take prostitutes back to their rooms and have some fun on D.C.’s dime, too?

Any wonder that Gallup reported on Sept. 8 that “only 8 percent of the one-third of all Americans who are following national politics ‘very closely’ approve of the way Congress is handling its job.” As Jon Stewart noted: “Here’s how dysfunctional the Secret Service is at this point, Congress had to help them come up with solutions.”

I can’t put my finger on it exactly, but you feel today in Washington a certain laxness, that anything goes and that too few people working for the federal government take pride in their work because everything is just cobbled together by Congress and the White House at the 11th hour anyway. It’s been years since anyone summoned us for a moonshot, for something great. So just show up and punch the clock.

In December 2010, I went to the White House for an interview. I entered through the Secret Service checkpoint on Pennsylvania Avenue. After putting my briefcase through the X-ray machine and collecting it, I grabbed the metal door handle to enter the White House driveway. The handle came off in my hand.

“Oh, it does that sometimes,” the Secret Service agent at the door said to me nonchalantly, as I tried to fit the wobbly handle back into the socket. People who take pride in their work don’t just let the handle come off a White House door like that.

Again, I’m not excusing the Secret Service, but the recent breakdowns don’t surprise me when so much of the political class that oversees the service is so self-absorbed, risk-averse and shortsighted. When the people governing us become this cynical, polarized and dysfunctional, it surely seeps down into the bureaucracy. As above, so below.""",
"""WHAT’S the right strategy for dealing with a world increasingly divided between zones of order and disorder? For starters, you’d better understand the forces of disorder, like Boko Haram or the Islamic State. These are gangs of young men who are telling us in every way possible that our rules no longer apply. Reason cannot touch them, because rationalism never drove them. Their barbarism comes from a dark place, where radical Islam gives a sense of community to humiliated, drifting young men, who have never held a job or a girl’s hand. That’s a toxic mix.

It’s why Orit Perlov, an Israeli expert on Arab social networks, keeps telling me that since I can’t visit the Islamic State, which is known as ISIS, and interview its leaders, the next best thing would be to see “Batman: The Dark Knight.” In particular, she drew my attention to this dialogue between Bruce Wayne and Alfred Pennyworth:

Bruce Wayne: “I knew the mob wouldn’t go down without a fight, but this is different. They crossed the line.”

Alfred Pennyworth: “You crossed the line first, sir. You squeezed them. You hammered them to the point of desperation. And, in their desperation, they turned to a man they didn’t fully understand.”

Bruce Wayne: “Criminals aren’t complicated, Alfred. Just have to figure out what he’s after.”

Alfred Pennyworth: “With respect, Master Wayne, perhaps this is a man that you don’t fully understand, either. A long time ago, I was in Burma. My friends and I were working for the local government. They were trying to buy the loyalty of tribal leaders by bribing them with precious stones. But their caravans were being raided in a forest north of Rangoon by a bandit. So we went looking for the stones. But, in six months, we never met anybody who traded with him. One day, I saw a child playing with a ruby the size of a tangerine. The bandit had been throwing them away.”

Bruce Wayne: “So why steal them?”

Alfred Pennyworth: “Well, because he thought it was good sport. Because some men aren’t looking for anything logical, like money. They can’t be bought, bullied, reasoned, or negotiated with. Some men just want to watch the world burn. ...”

Bruce Wayne: “The bandit, in the forest in Burma, did you catch him?”

Alfred Pennyworth: “Yes.”

Bruce Wayne: “How?”

Alfred Pennyworth: “We burned the forest down.”

We can’t just burn down Syria or Iraq or Nigeria. But there is a strategy for dealing with the world of disorder that I’d summarize with this progression:

Where there is disorder — think Libya, Iraq, Syria, Mali, Chad, Somalia — collaborate with every source of local, regional and international order to contain the virus until the barbarism burns itself out. These groups can’t govern, so ultimately locals will seek alternatives.

Where there is top-down order — think Egypt or Saudi Arabia — try to make it more decent and inclusive.

Where there is order plus decency — think Jordan, Morocco, Kurdistan, the United Arab Emirates — try to make it more consensual and effective, again to make it more sustainable.

Where there is order plus democracy — think Tunisia — do all you can to preserve and strengthen it with financial and security assistance, so it can become a model for emulation by the states and peoples around it.

And be humble. We don’t have the wisdom, resources or staying power to do anything more than contain these organisms, until the natural antibodies from within emerge.

In the Arab world, it may take longer for those natural antibodies to coalesce, and that is worrying, argues Francis Fukuyama, the Stanford political scientist whose new, widely discussed book, “Political Order and Political Decay,” is a historical study of how decent states emerge. What they all have in common is a strong and effective state bureaucracy that can deliver governance, the rule of law and regular rotations in power.

Because our founding fathers were escaping from tyranny, they were focused “on how power can be constrained,” Fukuyama explained to me in an interview. “But before power can be constrained, it has to be produced. ... Government is not just about constraints. It’s about providing security, infrastructure, health and rule of law. And anyone who can deliver all of that” — including China — “wins the game whether they are democratic or not. ... ISIS got so big because of the failure of governance in Syria and Iraq to deliver the most basic services. ISIS is not strong. Everything around it was just so weak,” riddled with corruption and sectarianism.

There is so much state failure in the Arab world, argues Fukuyama, because of the persistence there of kinship/tribal loyalties — “meaning that you can only trust that narrow group of people in your tribe.” You can’t build a strong, impersonal, merit-based state when the only ties that bind are shared kin, not shared values. It took China and Europe centuries to make that transition, but they did. If the Arab world can’t overcome its tribalism and sectarianism in the face of ISIS barbarism, “then there is nothing we can do,” said Fukuyama. And theirs will be a future of many dark nights.""",
"""I’ve been arguing for a while now that the Israeli-Palestinian conflict is to the wider East-West clash of civilizations what Off Broadway is to Broadway. It’s where you can see many trends at a smaller scale first. That is why I study it closely. Whether it is airline-hijacking, suicide-bombing or trying to do nation-building with the other — Israelis called it “Lebanon invasion” and “Oslo”; we called it “Iraq” and “Afghanistan” — what happens there often moves to the larger stage. So, as I have asked before: What’s playing Off Broadway now?

It’s a play called “Containment.” When faced with a barrage of rockets from the Hamas militants in Gaza, Israel largely retaliated with artillery and air power. These inflicted enough pain on Hamas and the Gaza civilian population that Hamas eventually agreed to a cease-fire — but not to surrender.

Indeed, Israel chose to deliberately leave Hamas in power in Gaza because it did not want to put Israeli boots on the ground and try to destroy it — which would have required bloody house-to-house fighting — and because Israel also did not want to leave Gaza as an ungoverned space. Israel’s adopting a strategy of containment toward Gaza also became viable after Egypt’s top military commander, Abdel Fattah el-Sisi, replaced the Muslim Brotherhood government led by Mohamed Morsi. The Brotherhood saw Hamas as an ally and allowed it to dig tunnels into Egypt and smuggle in goods for profit and rockets to hit Israel. Sisi, who sees the Brotherhood as his archenemy, has closed those tunnels.

So containment, as a purely military strategy to stem disorder, can work for Israel, for now. Containment also seems to be where the U.S.-led coalition is heading, for now, against the Islamic State, which is also known as ISIS. Since neither we nor our coalition partners are willing — or, thus far, in the case of the Iraqi Army and Syrian “moderates,” able — to put many boots on the ground to oust ISIS, we will rely on air power to prevent ISIS from expanding and maybe to shrink it.

But here we come to the most important difference between the containment we used to defeat the Soviet Union and the containment of Hamas and ISIS. We and the Israelis are both using containment to seal off a problem that we each perceive as too costly, politically and in human terms, to try to eliminate. But that strategy has its limits.

As Mark Mykleby, a retired Marine colonel and the co-director of the Strategic Innovation Lab at Case Western Reserve University, put it to me: “In the Cold War, we contained the Soviets militarily to set the conditions for the U.S.S.R. to collapse on itself, but that wasn’t the whole story. We also rebuilt the shattered economies of our former enemies, built international institutions like the I.M.F. and World Bank, and redesigned our own governing institutions to address our new post-World War II reality so that we would have the strategic scaffolding in place to continue building a post-Cold War world once the Soviet Union did in fact collapse.

“In the case of Gaza,” he added, “the Israelis are creating a self-fulfilling prophecy of a chronic problem by simply ‘containing’ the Palestinians.” Without a strategy for improving living conditions there, that could prove very damaging to Israel in the long run as Gaza becomes a human disaster zone. The West is doing something similar with ISIS: containing without building “the regional scaffolding to support and leverage” a more modern, consensual and pluralistic Middle East that might fill the ISIS space.

Containment, said Mykleby, only makes long-term sense if you commit money and political capital to fill that space with something decent. Israel is not doing that because Hamas refuses to recognize Israel’s right to exist and be a partner to a two-state solution. And because right-wing Jewish settlers so dominate Israel’s ruling coalition that Prime Minister Bibi Netanyahu cannot or will not put on the table anything close to what the more moderate Palestinian Authority demands for a two-state deal. Nor is it clear the Palestinians could deliver the security Israel demands. In short, the whole relationship is broken, making a strategy beyond containment very hard.

On Broadway, we’re hamstrung in building a post-ISIS political strategy by the fact that some of our coalition partners have no shared vision for a post-ISIS Syria or Iraq and do not want democracy in this region. Also, some of them, like Saudi Arabia and Qatar, are purveyors of the intolerant, anti-pluralistic Sunni ideology that inspires ISIS fighters. Even Turkey’s Islamist government has some pro-ISIS sympathies.

In short, containment in both theaters is necessary but not sufficient for long-term stability. But, unlike the Cold War where our containment strategy was largely the product of like-minded democracies working to liberate like-minded people from a bad system, in the Middle East, we have few like-minded partners.

The most we can hope for are “least bad” allies and “least bad” outcomes. In today’s Middle East, least bad is the new good.""",
"""OVER the past few weeks I’ve been reading Ken Adelman’s fascinating history “Reagan at Reykjavik: Forty-Eight Hours That Ended the Cold War.” Adelman, who led Reagan’s arms control agency, was an adviser at Reagan’s 1986 Iceland summit meeting with Soviet President Mikhail Gorbachev. Using some newly declassified documents, Adelman fills out the extraordinary dialogue between the two leaders that set in motion a dramatic cut in nuclear arms.

You learn a lot about Reagan’s leadership in the book. For me, the most impressive thing was not Reagan’s attachment to his “Star Wars” strategic defense initiative, which is overrated in ending the Cold War. What is most impressive about Reagan is that he grasped that Gorbachev was a radically different kind of Soviet leader — one with whom he could make history — long before his intelligence community did. That made a big difference.

These days there is a lot of “if-only-Obama-could-lead-like-Reagan” talk by conservatives. I’ll leave it to historians to figure out years from now who was the better president. But what I’d argue is this: In several critical areas, Reagan had a much easier world to lead in than Obama does now.
“Easier world, are you kidding?” say conservatives. “Reagan was up against a Communist superpower that had thousands of nuclear missiles aimed at us! How can you say that?”

Here’s how: The defining struggle in Reagan’s day was the Cold War, and the defining feature of the Cold War was that it was a war between two different systems of order: Communism versus democratic capitalism. But both systems competed to build order — to reinforce weak states around the world with military and economic aid and win their support in the Cold War. And when either Moscow or Washington telephoned another state around the world, there was almost always someone to answer the phone. They even ensured that their proxy wars — like Vietnam and Afghanistan — were relatively contained.

Obama’s world is different. It is increasingly divided by regions of order and regions of disorder, where there is no one to answer the phone, and the main competition is not between two organized superpowers but between a superpower and many superempowered angry men. On 9/11, we were attacked, and badly hurt, by a person: Osama bin Laden, and his superempowered gang. When superempowered angry men have more open space within which to operate, and more powerful weapons and communication tools, just one needle in a haystack can hurt us.

Most important, Reagan’s chief rival, Gorbachev, won the Nobel Peace Prize in 1990 for doing something he never wanted to do: peacefully letting go of Eastern Europe. Obama’s foes, like the Islamic State, will never win the Nobel Peace Prize. Reagan could comfortably challenge Gorbachev in Berlin to “tear down this wall” because on the other side of that wall was a bad system — Communism — that was suppressing a civilization in Eastern and Central Europe, and part of Russia, that was naturally and historically inclined toward democratic capitalism. And there were leaders there — like Lech Walesa, another Nobel Peace Prize winner — to lead the transition. We just needed to help remove the bad system and step aside.

“The countries of Eastern and Central Europe were forcibly part of a Communist empire but culturally were always part of Western civilization,” explained Michael Mandelbaum, the Johns Hopkins University foreign policy specialist and author of “The Road to Global Prosperity.” “They never saw themselves as Communist, but rather as Westerners who had been kidnapped.” After Gorbachev, under pressure from Reagan and the West, released them, “they ran as fast as they could to embrace Western institutions.”

In the Middle East, which has consumed so much of Obama’s energy, the people tore down their walls — their systems — but underneath was not a civilization with the suppressed experience, habits and aspirations of democracy and free markets. Instead it was a toxic mix of Islamism, tribalism, sectarianism and an inchoate aspiration for democracy.

Reagan’s leadership challenge was to bring down a wall and then reap the peace dividends by just letting nature take its course. Obama’s challenge is that on the other side of the wall that the Arabs took down lies the world’s biggest nation-building project, with a civilization that is traumatized, divided and often culturally hostile to Western values and institutions. It’s an enormous job that only the locals can lead.

The one time that Reagan faced the miniversion of Obama’s challenge was in Lebanon. After Israel toppled the Palestinian ministate there, Reagan hoped it would unleash a naturally democratic order, with just a little midwifing help from American Marines. But after 241 U.S. servicemen were blown up in Beirut in 1983, Reagan realized that the civilization there was a mix of Islamists, sectarian Christians, Syrians, Shiite militias, Palestinian refugees and democrats. It required a lot more than us just standing guard. It required nation-building. And what did Reagan do? He left.

I was there to wave goodbye to the last Marines on the beaches of Beirut.

So comparing Reagan with Obama in foreign policy is inevitable. But when you do, also compare their respective contexts. The difference is revealing.""",
"""There is a tension at the heart of President Obama’s campaign to confront the Islamic State, and it explains a lot about why he has so much trouble articulating and implementing his strategy. Quite simply, it is the tension between two vital goals — promoting the “soul-searching” that ISIS’s emergence has triggered in the Arab-Muslim world and “searching and destroying” ISIS in its strongholds in Syria and Iraq.

Get used to it. This tension is not going away. Obama will have to lead through it.

The good news: The rise of the Islamic State, also known as ISIS, is triggering some long overdue, brutally honest, soul-searching by Arabs and Muslims about how such a large, murderous Sunni death cult could have emerged in their midst. Look at a few samples, starting with “The Barbarians Within Our Gates,” written in Politico last week by Hisham Melhem, the Washington bureau chief of Al-Arabiya, the Arabic satellite channel.

Continue reading the main story
RELATED IN OPINION

Editorial: Wrong Turn on Syria: No Convincing PlanSEPT. 23, 2014
Editorial: Wrong Turn on Syria: Helping Assad?SEPT. 23, 2014
Strikes on ISIS. Now What?SEPT. 23, 2014
Op-Ed Contributor: The Ancestors of ISISSEPT. 23, 2014
“With his decision to use force against the violent extremists of the Islamic State, President Obama ... is stepping once again — and with understandably great reluctance — into the chaos of an entire civilization that has broken down. Arab civilization, such as we knew it, is all but gone. The Arab world today is more violent, unstable, fragmented and driven by extremism — the extremism of the rulers and those in opposition — than at any time since the collapse of the Ottoman Empire a century ago.

“Every hope of modern Arab history has been betrayed,” Melhem added. “The promise of political empowerment, the return of politics, the restoration of human dignity heralded by the season of Arab uprisings in their early heydays — all has given way to civil wars, ethnic, sectarian and regional divisions and the reassertion of absolutism, both in its military and atavistic forms. ... The jihadists of the Islamic State, in other words, did not emerge from nowhere. They climbed out of a rotting, empty hulk — what was left of a broken-down civilization.”

The liberal Saudi analyst Turki al-Hamad responded in the London-based Al-Arab newspaper to King Abdullah’s call for Saudi religious leaders to confront ISIS ideology: How can they? al-Hamad asked. They all embrace the same anti-pluralistic, puritanical Wahhabi Sunni ideology that Saudi Arabia diffused, at home and abroad, to the mosques that nurtured ISIS.

“They are unable to face the groups of violence, extremism and beheadings, not out of laziness or procrastination, but because all of them share in that same ideology,” al-Hamad wrote. “How can they confront an ideology that they themselves carry within them and within their mind-set?”

The Lebanese Shiite writer Hanin Ghaddar in an essay in August on Lebanon’s Now website wrote: “To fight the I.S. and other radical groups, and to prevent the rise of new autocratic rulers, we need to assume responsibility for the collective failures that have produced all of these awful tyrants and fanatics. Our media and education systems are liable for the monster we helped create. ... We need to teach our children how to learn from our mistakes instead of how to master the art of denial. When our educators and journalists start to understand the significance of individual rights, and admit that we have failed to be citizens, then we can start hoping for freedom, even if it is achieved slowly.”

Nurturing this soul-searching is a vital — and smart — part of the Obama strategy. In committing America to an air-campaign-only against ISIS targets in Syria and Iraq, Obama has declared that the ground war will have to be fought by Arabs and Muslims, not just because this is their war and they should take the brunt of the casualties, but because the very act of their organizing themselves across Shiite, Sunni and Kurdish lines — the very act of overcoming their debilitating sectarian and political differences that would be required to defeat ISIS on the ground — is the necessary ingredient for creating any kind of decent, consensual government that could replace ISIS in any self-sustaining way.

The tension arises because ISIS is a killing machine, and it will take another killing machine to search it out and destroy it on the ground. There is no way the “moderate” Syrians we’re training can alone fight ISIS and the Syrian regime at the same time. Iraqis, Turkey and the nearby Arab states will have to also field troops.

After all, this is a civil war for the future of both Sunni Islam and the Arab world. We can degrade ISIS from the air — I’m glad we have hit these ISIS psychopaths in Syria — but only Arabs and Turks can destroy ISIS on the ground. Right now, Turkey’s president, Recep Tayyip Erdogan, stands for authoritarianism, press intimidation, crony capitalism and quiet support for Islamists, including ISIS. He won’t even let us use our base in Turkey to degrade ISIS from the air. What’s in his soul? What’s in the soul of the Arab regimes who are ready to join us in bombing ISIS in Syria, but rule out ground troops?

This is a civilization in distress, and unless it faces the pathologies that have given birth to an ISIS monster its belly — any victory we achieve from the air or ground will be temporary.""",
"""MADRID — THIS was an interesting week to visit Britain and Spain — first to watch the Scottish separatists push for independence and then to watch Basque and Catalan separatists watching (with disappointment) the outcome of the vote. One reaction: I’m glad a majority of Scots rejected independence. Had they not, it would have clipped the wing of America’s most important wingman in the world: Britain. Another reaction: God bless America. We have many sources of strength, but today our greatest asset is our pluralism — our “E pluribus unum” — that out of many we’ve made one nation, with all the benefits that come from mixing cultures and all the strengths that come from being able to act together.

As I’ve asked before: Who else has twice elected a black man as president, whose middle name is Hussein, whose grandfather was a Muslim, who first defeated a woman and later defeated a Mormon? I’m pretty sure that I will not live long enough to see an ethnic Pakistani become prime minister of Britain or a Moroccan immigrant president of France. Yes, the unrest in Ferguson, Mo., reminds us that we’re still a work in progress in the pluralism department. But work on it we do, and I’ll take the hard work of pluralism over the illusions of separatism any day.

Why is pluralism such a big advantage today? Two reasons: politics and innovation. Before I explain, though, it’s worth recalling: What is pluralism? I like the definition that the Pluralism Project at Harvard offers on its website: “pluralism is not diversity alone, but the energetic engagement with diversity” because “mere diversity without real encounter and relationship will yield increasing tensions in our societies.” A society being “pluralistic” is a reality (see Syria and Iraq). A society with pluralism “is an achievement” (see America).

Pluralism, it also notes, “does not require us to leave our identities and our commitments behind. ... It means holding our deepest differences, even our religious differences, not in isolation, but in relationship to one another.” And, it posits that real pluralism is built on “dialogue” and “give and take, criticism and self-criticism” — and “dialogue means both speaking and listening.”

That pluralism is more important than ever is easily divined by just looking at the Middle East. Iraq and Syria were pluralistic societies that lacked pluralism. Their diversity — Sunnis, Shiites, Kurds, Turkmen, Christians, Jews, Yazidis, Alawites — was something to be controlled from the top down by iron-fisted Ottomans, then the British and French and finally by local kings and colonels. Society was kept stable by a strongman.

But the diffusion of communication technologies and globalization is making all forms of top-down, autocratic control weaker, obsolete or more expensive in blood, money or arrests. Either these countries develop an ethic of pluralism — so they can govern themselves horizontally through social contracts forged among equal citizens — or they’ll stay in violent turmoil.

It’s no accident that the two democratizing Middle East entities doing best today are Tunisia and Kurdistan. Neither has fully mastered pluralism yet, but they’ve mastered its necessary precursor for self-governance, which was the principle used in 1989 to settle the Lebanese civil war: “No victor, no vanquished” among the major players. Everyone’s interests have to be balanced. Iraq is now struggling to get there; Syria is not even close.

Social networks and hyperglobalization are also increasing the economic returns from pluralism. After all, where does innovation come from? It comes from mashing up different perspectives, ideas and people. Google began as a mashup between Larry Page and Sergey Brin, a Russian immigrant. The more pluralism your society has, the more trust it has, and trust plus pluralism enables people to collaborate, spark new ideas and businesses, and to comfortably reach out anywhere in the globe for the best co-creators. Sure, melting pots can boil over, but, when fueled by a pluralistic ethic, the energy they provide is undeniable. The Economist reported in April 2013 that some “40 percent of Fortune 500 firms were founded by immigrants or their children.”

Democratic Spain in the last decade has impressively absorbed more than four million immigrants — mostly from Ecuador, Romania and Morocco — or 10 percent of its population. They came during the economic boom and have triggered no anti-immigrant party (yet). No wonder Spain’s national leaders today expressed relief at the no vote in Scotland. But the Catalan regional government insists it will proceed with its own nonbinding separatist referendum in November.

That will meet headwinds. To manage its diversity, Spain already awards a lot of autonomy to its 17 regions — a process called “coffee for all” — and many Spaniards “don’t want” to be pressed into a deeper breakup, explained José Ignacio Torreblanca, the head of the Madrid office of the European Council on Foreign Relations. “You go to Barcelona and people are hanging the Catalan independence flag on their balcony. If you’re not, it means you’re not in favor of independence, but I don’t want to fight you by hanging the Spanish flag.” Many people here think you can be “a good Spaniard, good Catalan and good European” all at once.

The other danger of all these separatist movements, added Torreblanca, is that they “change the axis” of the political debate. “Politics should be about left and right — how to grow and how to redistribute.” Historically in Europe, he said, right-wing parties come in and create growth and inequality and left-wing parties come in and redistribute — and back and forth. “But the net result is that you end up with societies that are both competitive and cohesive.” All these separatist movements take you off that track, he said, and put you onto one of “identity politics,” which is precisely why places like Syria and Iraq can’t make progress.

AMERICA has always been “a country of citizens,” which made its pluralism relatively easy, noted Torreblanca. “The Europe Union is a country of nation states,” and it is trying to get more pluralistic by integrating those states ever more tightly into a super-state, called the European Union. But that is stalled now because the next level of integration requires not just giving up your currency but sovereignty, so there can be a truly common economic policy. In Syria and Iraq today, you have neither citizens nor states, but rather clans, sects and tribes, which now need to reorganize themselves into voluntary states, as opposed to those imposed by colonial powers, so they can be real citizens.

This is why America has such an advantage with its pluralism, and why — if Scots are brave enough to preserve theirs, and Spaniards are struggling to keep theirs and Iraqis are groping to find theirs — we should have the wisdom to pass an immigration reform bill that enriches ours.""",
"""LONDON — An existential struggle is taking place in the Arab world today. But is it ours or is it theirs? Before we step up military action in Iraq and Syria, that’s the question that needs answering.

What concerns me most about President Obama’s decision to re-engage in Iraq is that it feels as if it’s being done in response to some deliberately exaggerated fears — fear engendered by YouTube videos of the beheadings of two U.S. journalists — and fear that ISIS, a.k.a., the Islamic State, is coming to a mall near you. How did we start getting so afraid again so fast? Didn’t we build a Department of Homeland Security?

I am not dismissing ISIS. Obama is right that ISIS needs to be degraded and destroyed. But when you act out of fear, you don’t think strategically and you glide over essential questions, like why is it that Shiite Iran, which helped trigger this whole Sunni rebellion in Iraq, is scoffing at even coordinating with us, and Turkey and some Arab states are setting limits on their involvement?

When I read that, I think that Nader Mousavizadeh, who co-leads the global consulting firm Macro Advisory Partners, is correct when he says: “When it comes to intervening in the Arab world’s existential struggle, we have to stop and ask ourselves why we have such a challenge getting them to help us save them.”

So before we get in any deeper, let’s ask some radical questions, starting with: What if we did nothing? George Friedman (no relation), the chairman of Stratfor, raised this idea in his recent essay on Stratfor.com, “The Virtue of Subtlety.” He notes that the ISIS uprising was the inevitable Sunni backlash to being brutally stripped of power and resources by the pro-Iranian Shiite governments and militias in Baghdad and Syria. But then he asks:

Is ISIS “really a problem for the United States? The American interest is not stability but the existence of a dynamic balance of power in which all players are effectively paralyzed so that no one who would threaten the United States emerges. ... But the principle of balance of power does not mean that balance must be maintained directly. Turkey, Iran and Saudi Arabia have far more at stake in this than the United States. So long as they believe that the United States will attempt to control the situation, it is perfectly rational for them to back off and watch, or act in the margins, or even hinder the Americans. The United States must turn this from a balance of power between Syria and Iraq to a balance of power among this trio of regional powers. They have far more at stake and, absent the United States, they have no choice but to involve themselves. They cannot stand by and watch a chaos that could spread to them.”

Therefore, he concludes, the best U.S. strategy rests in us “doing as little as possible and forcing regional powers into the fray, then in maintaining the balance of power in this coalition.” I am not sure, but it’s worth debating.

Here’s another question: What’s this war really about?

“This is a war over the soul of Islam — that is what differentiates this moment from all others,” argues Ahmad Khalidi, a Palestinian scholar associated with St. Antony’s College, Oxford. Here is why: For decades, Saudi Arabia has been the top funder of the mosques and schools throughout the Muslim world that promote the most puritanical version of Islam, known as Salafism, which is hostile to modernity, women and religious pluralism, or even Islamic pluralism.

Saudi financing for these groups is a byproduct of the ruling bargain there between the al-Saud family and its Salafist religious establishment, known as the Wahhabis. The al-Sauds get to rule and live how they like behind walls, and the Wahhabis get to propagate Salafist Islam both inside Saudi Arabia and across the Muslim world, using Saudi oil wealth. Saudi Arabia is, in effect, helping to fund both the war against ISIS and the Islamist ideology that creates ISIS members (some 1,000 Saudis are believed to be fighting with jihadist groups in Syria), through Salafist mosques in Europe, Pakistan, Central Asia and the Arab world.

This game has reached its limit. First, because ISIS presents a challenge to Saudi Arabia. ISIS says it is the “caliphate,” the center of Islam. Saudi Arabia believes it is the center. And, second, ISIS is threatening Muslims everywhere. Khalidi told me of a Muslim woman friend in London who says she’s afraid to go out with her head scarf on for fear that people will believe she is with ISIS — just for dressing as a Muslim. Saudi Arabia cannot continue fighting ISIS and feeding the ideology that nurtures ISIS. It will hurt more and more Muslims.

We, too, have to stop tolerating this. For years, the U.S. has “played the role of the central bank of Middle East stability,” noted Mousavizadeh. “Just as the European Central Bank funding delays the day that France has to go through structural reforms, America’s security umbrella,” always there no matter what the Saudis do, “has delayed the day that Saudi Arabia has to face up to its internal contradictions,” and reform its toxic ruling bargain. The future of Islam and our success against ISIS depend on it."""
]

### Filling Mongo Database with Post

In [12]:
# Blogs in MongoDB

blogs = [
            ['http://www.rednecksense.blogspot.com/', 'conservative', 152],
            ['http://richardschmitt.blogspot.com/', 'liberal', 348],
            ['http://chuckmarshall.blogspot.com/', 'moderate', 122],
            ['http://agoodchoice.blogspot.com/', 'conservative', 4142],
            ['http://pbd.blogspot.com/', 'liberal', 3745],
            ['http://knappster.blogspot.com/', 'liberal', 2529],
            ['http://snunes.blogspot.com/', 'liberal', 42222],
            ['http://c-pol.blogspot.com/', 'conservative', 919],
            ['http://apologiesdemanded.blogspot.com/', 'conservative', 2847],
            ['http://jdeeth.blogspot.com/', 'moderate', 6167],
            ['http://redstatediaries.blogspot.com/', 'moderate', 1397],
            ['http://www.shakesville.com/', 'liberal', 38897],   
            ['http://plainblogaboutpolitics.blogspot.com/', 'review', 4984],
            ['http://2politicaljunkies.blogspot.com/', 'review', 6682],
            ['http://googlepolitics.blogspot.com/', 'review', 145],
        ]

In [51]:
for blog in blogs:
    out = list(poliblogs.find({'blog_url': blog[0] }, {'_id'}))
    post_num = len(out)
    blog.append(post_num)
    
#     http://snunes.blogspot.com/ http://www.shakesville.com/

[['http://www.rednecksense.blogspot.com/', 'conservative', 152], ['http://richardschmitt.blogspot.com/', 'liberal', 348], ['http://chuckmarshall.blogspot.com/', 'moderate', 122], ['http://agoodchoice.blogspot.com/', 'conservative', 4142], ['http://pbd.blogspot.com/', 'liberal', 3745], ['http://knappster.blogspot.com/', 'liberal', 2529], ['http://snunes.blogspot.com/', 'liberal', 42222], ['http://c-pol.blogspot.com/', 'conservative', 919], ['http://apologiesdemanded.blogspot.com/', 'conservative', 2847], ['http://jdeeth.blogspot.com/', 'moderate', 6167], ['http://redstatediaries.blogspot.com/', 'moderate', 1397], ['http://www.shakesville.com/', 'liberal', 38897], ['http://plainblogaboutpolitics.blogspot.com/', 'review', 4984], ['http://2politicaljunkies.blogspot.com/', 'review', 6682], ['http://googlepolitics.blogspot.com/', 'review', 145]]


In [17]:
# Actually insert blogs
# for b in blogs: 
#     blog_posts = tscrape.get_articles_blogger(b[0], google_api_key)
#     insert_posts_in_mongo(blog_posts, b[1], poliblogs)

Number of blog posts inserted into Mongo: 152
Number of blog posts inserted into Mongo: 348
Number of blog posts inserted into Mongo: 122
Number of blog posts inserted into Mongo: 4142
Number of blog posts inserted into Mongo: 3745
Number of blog posts inserted into Mongo: 2529
Number of blog posts inserted into Mongo: 42226
Number of blog posts inserted into Mongo: 919

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://evilhrlady.blogspot.com/2007/03/responsibility-vs-authority.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://photos-e.ak.fbcdn.net/hphotos-ak-snc1/hs175.snc1/6570_911014557384_2311632_55141752_7738360_n.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful S


Number of blog posts inserted into Mongo: 2847
Number of blog posts inserted into Mongo: 6167
Number of blog posts inserted into Mongo: 1397
Number of blog posts inserted into Mongo: 38906
Number of blog posts inserted into Mongo: 4984
Number of blog posts inserted into Mongo: 6682
Number of blog posts inserted into Mongo: 145


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://74.52.106.178:7350/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


In [39]:
# # Go back through and add number of sentences as a field
fullcollect  = poliblogs.find({}, {"post_content":1})
for post in fullcollect:
    num_sentences = len(sent_tokenize(post["post_content"]))       
    poliblogs.update({"_id": post["_id"]}, {"$set": {"num_sentences": num_sentences}})

In [18]:
# nltk.download()

In [35]:
poliblogs.count()

115311

In [15]:
# poliblogs.remove({})

{u'n': 4764, u'ok': 1}

### Random Snips

In [121]:
## Based on using entire blog to predict. 
## I decided this was not as effective due to the low number of blogs I have to train on.

# # Get random sample of posts from each blog
# blogsample_nums = {}
# for blog in blogs:
#     nsample = 1000
#     out = list(poliblogs.find({'blog_url': blog[0]}, {'_id':1}))
#     length = len(out)
#     if nsample > length:
#         nsample = length
#     print("For %s, sample is %i/%i" % (blog[0], nsample, length))
#     out = random.sample(out, nsample)
#     blogsample_nums.update({blog[0]: out})
   

# # And build list of categories of blogs
# blogsample_categories = [blog[1] for blog in blogs]

# # Actually grab and concatenate blog posts
# blogsamples = {}
# for blog in blogsample_nums.keys():
#     post_nums = [p['_id'] for p in blogsample_nums[blog]]
#     blog_texts = [p['post_content'] for thisid in post_nums for p in poliblogs.find({'_id': thisid})]
#     concat = ' '.join(blog_texts)
#     blogsamples[blog] = concat

# Stupid testing
# testsample_nums = {}
# testsample_categories = []
# for blog in blogs:
#     nsample = 500
#     out = list(poliblogs.find({'blog_url': blog[0]}, {'_id':1}))
#     length = len(out)
#     if nsample < 0.2*length:
#         print("For %s, sample is %i/%i" % (blog[0], nsample, length))
#         out = random.sample(out, nsample)
#         testsample_nums.update({blog[0]: out})
        
#         # And build list of categories of blogs
#         testsample_categories.append(blog[1])

# # Actually grab and concatenate blog posts
# testsamples = {}
# for blog in testsample_nums.keys():
#     post_nums = [p['_id'] for p in blogsample_nums[blog]]
#     blog_texts = [p['post_content'] for thisid in post_nums for p in poliblogs.find({'_id': thisid})]
#     concat = ' '.join(blog_texts)
#     testsamples[blog] = concat